In [1]:
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers
from keras.callbacks import EarlyStopping
import random
import numpy as np
from tensorflow import keras

In [2]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [3]:
# Normalize pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [4]:
# Convert labels to categorical one-hot encoding
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [5]:
# Split the training dataset into validation and partial training sets
val_images = X_train[:10000]
partial_images = X_train[10000:]

val_labels = y_train[:10000]
partial_labels = y_train[10000:]

In [6]:
class GeneticCNNOptimizer:
    def __init__(self, partial_images, partial_labels, val_images, val_labels, X_test, y_test, population_size=10, generations=3, threshold=90):
        self.partial_images = partial_images
        self.partial_labels = partial_labels
        self.val_images = val_images
        self.val_labels = val_labels
        self.X_test = X_test
        self.y_test = y_test
        self.population_size = population_size
        self.generations = generations
        self.threshold = threshold
        self.population = self.generate_population(population_size)

    def create_custom_cnn_model(self, chromosome):
        custom_model = models.Sequential()
        custom_model.add(layers.Conv2D(filters=chromosome["filters_1"], kernel_size=chromosome["kernel_size"], activation=chromosome["activation_1"], input_shape=(32, 32, 3)))
        custom_model.add(layers.Conv2D(filters=chromosome["filters_1"], kernel_size=chromosome["kernel_size"], activation=chromosome["activation_1"]))
        custom_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        custom_model.add(layers.Conv2D(filters=chromosome["filters_2"], kernel_size=chromosome["kernel_size"], activation=chromosome["activation_2"]))
        custom_model.add(layers.Conv2D(filters=chromosome["filters_2"], kernel_size=chromosome["kernel_size"], activation=chromosome["activation_2"]))
        custom_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        custom_model.add(layers.Flatten())
        custom_model.add(layers.Dropout(rate=chromosome["dropout_1"]))
        custom_model.add(layers.Dense(units=chromosome["filters_3"], activation=chromosome["activation_2"]))
        custom_model.add(layers.Dropout(rate=chromosome["dropout_2"]))
        custom_model.add(layers.Dense(units=10, activation="softmax"))

        optimizer = getattr(optimizers, chromosome["optimizer"])()
        custom_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        early_stopping = EarlyStopping(monitor="val_accuracy", patience=7)
        custom_model.fit(self.partial_images, self.partial_labels, validation_data=(self.val_images, self.val_labels), epochs=chromosome["epochs"], batch_size=100, callbacks=[early_stopping], verbose=0)

        return custom_model

    def initialize_parameters(self):
        parameters = {
            "filters_1": random.choice([32, 64]),
            "filters_2": random.choice([64, 128]),
            "filters_3": random.choice([128, 256, 512]),
            "kernel_size": random.choice([(3, 3), (5, 5)]),
            "activation_1": random.choice(["relu", "sigmoid", "tanh"]),
            "activation_2": random.choice(["relu", "sigmoid", "tanh"]),
            "dropout_1": round(random.uniform(0.1, 0.5), 1),
            "dropout_2": round(random.uniform(0.1, 0.5), 1),
            "optimizer": random.choice(["Adam", "Adagrad", "SGD", "RMSprop"]),
            "epochs": np.random.randint(50, 80)
        }
        return parameters

    def generate_population(self, n):
        return [self.initialize_parameters() for _ in range(n)]

    def evaluate_fitness(self, model):
        evaluation_metrics = model.evaluate(self.X_test, self.y_test, verbose=0)
        accuracy = evaluation_metrics[1]
        return accuracy

    def selection(self, population_fitness):
        total = sum(population_fitness)
        percentage = [round((x / total) * 100) for x in population_fitness]
        selection_wheel = []
        for pop_index, num in enumerate(percentage):
            selection_wheel.extend([pop_index] * num)
        parent1_ind = random.choice(selection_wheel)
        parent2_ind = random.choice(selection_wheel)
        return [parent1_ind, parent2_ind]

    def crossover(self, parent1, parent2):
        child1, child2 = {}, {}
        for key in parent1:
            child1[key], child2[key] = random.choice([(parent1[key], parent2[key]), (parent2[key], parent1[key])])
        return [child1, child2]

    def mutation(self, chromosome):
        if random.randint(0, 1):
            chromosome["epochs"] = min(100, chromosome["epochs"] + random.randint(1, 10))
        return chromosome

    def evolve(self):
        for generation in range(self.generations):
            print(f"Starting Generation {generation + 1}/{self.generations}")

            # Evaluate the fitness of each individual in the population
            population_fitness = []
            for chromosome in self.population:
                try:
                    model = self.create_custom_cnn_model(chromosome)
                    fitness = self.evaluate_fitness(model)
                    population_fitness.append(fitness)
                    print(f"Chromosome: {chromosome}, Fitness (Accuracy): {fitness}")
                except Exception as e:
                    print(f"Error training model with chromosome {chromosome}: {e}")
                    population_fitness.append(0)  # Assign a low fitness value to failed models

            # Selection of parents for the next generation
            parents_indices = self.selection(population_fitness)
            parent1 = self.population[parents_indices[0]]
            parent2 = self.population[parents_indices[1]]

            # Crossover and mutation to create new offspring
            children = self.crossover(parent1, parent2)
            mutated_children = [self.mutation(child) for child in children]

            # Add new offspring to the population
            self.population.extend(mutated_children)

            # Remove least fit individuals
            sorted_fitness_indices = sorted(range(len(population_fitness)), key=lambda i: population_fitness[i])
            self.population = [self.population[i] for i in sorted_fitness_indices[-self.population_size:]]

            max_fitness = max(population_fitness)
            print(f"End of Generation {generation + 1}, Max Fitness (Accuracy): {max_fitness}")

            # Check if the desired threshold has been met
            if max_fitness >= self.threshold:
                print(f"Desired accuracy threshold of {self.threshold}% achieved. Stopping evolution.")
                break

In [7]:
# Create an instance of the GeneticCNNOptimizer class
hypertune = GeneticCNNOptimizer(
    partial_images=partial_images,
    partial_labels=partial_labels,
    val_images=val_images,
    val_labels=val_labels,
    X_test=X_test,
    y_test=y_test,
    population_size=5,
    generations=1,
    threshold=0.80  # This threshold is a proportion (e.g., 0.90 for 90% accuracy)
)

In [8]:
# Start the evolution process
hypertune.evolve()

Starting Generation 1/1
Chromosome: {'filters_1': 32, 'filters_2': 128, 'filters_3': 256, 'kernel_size': (5, 5), 'activation_1': 'sigmoid', 'activation_2': 'tanh', 'dropout_1': 0.5, 'dropout_2': 0.3, 'optimizer': 'RMSprop', 'epochs': 58}, Fitness (Accuracy): 0.10000000149011612
Chromosome: {'filters_1': 32, 'filters_2': 64, 'filters_3': 256, 'kernel_size': (5, 5), 'activation_1': 'relu', 'activation_2': 'relu', 'dropout_1': 0.2, 'dropout_2': 0.3, 'optimizer': 'Adam', 'epochs': 60}, Fitness (Accuracy): 0.7014999985694885
Chromosome: {'filters_1': 64, 'filters_2': 128, 'filters_3': 128, 'kernel_size': (3, 3), 'activation_1': 'relu', 'activation_2': 'tanh', 'dropout_1': 0.3, 'dropout_2': 0.3, 'optimizer': 'SGD', 'epochs': 71}, Fitness (Accuracy): 0.730400025844574
Chromosome: {'filters_1': 64, 'filters_2': 64, 'filters_3': 128, 'kernel_size': (3, 3), 'activation_1': 'sigmoid', 'activation_2': 'tanh', 'dropout_1': 0.2, 'dropout_2': 0.4, 'optimizer': 'SGD', 'epochs': 51}, Fitness (Accuracy)